In [24]:
%run -i writefile2.py

In [25]:
%%writefile2 --name server.py

from flask import Flask, render_template, send_from_directory
from flask_socketio import SocketIO
import os
import require
import functools
get_data_for_country = require.single( "get_data_for_country" )

app = Flask( __name__, static_folder = "static", static_url_path = '' )
socketio = SocketIO(app)

@functools.cache
def load_data( country ):

    print( f"loading data for { country }" )
    return get_data_for_country( country, verbose = False )

@app.route('/')
def serve_index():
    return send_from_directory('static', 'index.html')

@socketio.event
def my_event(json):
    print('received json: ' + str(json))
    return "OK", 200

@socketio.event
def get_data( country ):
    series = load_data( country ).new_cases_smoothed_per_million
    series.name = "close"
    return series.to_csv( )

def main( ):

    try:

        print( "running..." )
        print( "http://localhost:8080" )
        socketio.run( app, port = 8080, host = "0.0.0.0" )    

    except:
        
        print( "done." )

if "is_server" in os.environ:

    main( )